In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from torchinfo import summary
from models import XSwinFusion 
from pose_estimation import PoseDataNPZTorch


WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
x = torch.tensor([0, 1, 2, 3, 4, 5])

x = x[[2,3, 2,3,2 ,3 ,2 ,3]]
x


tensor([2, 3, 2, 3, 2, 3, 2, 3])

In [3]:
samples = 1_000
resize = (144, 256)
aspect_ratio = True
margin = 12
feature_dims = 64
pretrained = True

model = XSwinFusion(num_points=samples, feature_dims=feature_dims, resize=resize, pretrained=True)

dataset = None
dataset = PoseDataNPZTorch(DATASET_NPZ_PATH, samples=samples, 
                           resize=resize, aspect_ratio=aspect_ratio, margin=margin)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

summary(model, depth=1)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


Layer (type:depth-idx)                                       Param #
XSwinFusion                                                  --
├─FCN: 1-1                                                   35,355,328
├─PointNet: 1-2                                              622,482
├─Sequential: 1-3                                            12,672
├─Sequential: 1-4                                            265,088
├─Sequential: 1-5                                            3,161
├─Sequential: 1-6                                            2,883
├─Sequential: 1-7                                            2,865
Total params: 36,264,479
Trainable params: 36,260,374
Non-trainable params: 4,105

In [4]:
R = torch.eye(3)
T = torch.ones((3, 1))
print(R.shape, T.shape)
torch.cat((R, T), dim=-1)


torch.Size([3, 3]) torch.Size([3, 1])


tensor([[1., 0., 0., 1.],
        [0., 1., 0., 1.],
        [0., 0., 1., 1.]])

In [5]:
points = []

for i, (s, t, c, mi, p) in enumerate(dataloader):

    x, transforms = model(t, c, mi)
    print(s.shape)
    print(t.shape)
    print(x.shape)

    R = x[:, :, :3]
    T = x[:, :, 3]

    x = torch.bmm(s, R.transpose(-2, -1)) + T.unsqueeze(-2)
    print(x.shape)

    if i == 0: 
        assert 0


OrderedDict([('out', tensor([[[[ 0.0751,  0.0751,  0.0751,  ..., -0.0408, -0.0408, -0.0408],
          [ 0.0751,  0.0751,  0.0751,  ..., -0.0408, -0.0408, -0.0408],
          [ 0.0751,  0.0751,  0.0751,  ..., -0.0408, -0.0408, -0.0408],
          ...,
          [ 0.0919,  0.0919,  0.0919,  ...,  0.0997,  0.0997,  0.0997],
          [ 0.0919,  0.0919,  0.0919,  ...,  0.0997,  0.0997,  0.0997],
          [ 0.0919,  0.0919,  0.0919,  ...,  0.0997,  0.0997,  0.0997]],

         [[ 0.2080,  0.2080,  0.2080,  ...,  0.1585,  0.1585,  0.1585],
          [ 0.2080,  0.2080,  0.2080,  ...,  0.1585,  0.1585,  0.1585],
          [ 0.2080,  0.2080,  0.2080,  ...,  0.1585,  0.1585,  0.1585],
          ...,
          [ 0.1043,  0.1043,  0.1043,  ..., -0.0516, -0.0516, -0.0516],
          [ 0.1043,  0.1043,  0.1043,  ..., -0.0516, -0.0516, -0.0516],
          [ 0.1043,  0.1043,  0.1043,  ..., -0.0516, -0.0516, -0.0516]],

         [[-0.0849, -0.0849, -0.0849,  ..., -0.0560, -0.0560, -0.0560],
         

TypeError: permute(): argument 'input' (position 1) must be Tensor, not collections.OrderedDict